In [28]:
# ============================
# 1. Imports
# ============================
import os
import time
import math
import random
from dataclasses import dataclass
from typing import Optional, Tuple, Dict, List

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.transforms as T

In [29]:
# ============================
# 2. Config (Experiment Settings)
# ============================

from dataclasses import dataclass
from typing import Tuple
import torch

@dataclass
class Config:
    # --- data ---
    data_dir: str = "./data"
    batch_size: int = 128
    num_workers: int = 2   # Colab safe default
    num_classes: int = 100

    # --- training ---
    num_epochs: int = 120   # ✅ baseline 표준값 (200 → 120)
    lr: float = 0.1
    momentum: float = 0.9
    weight_decay: float = 5e-4
    seed: int = 42
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

    # --- logging ---
    exp_name: str = "cifar100_resnet18"
    variant: str = "baseline"  # baseline | shift_int | shift_grid | scale | ...
    log_root: str = "./runs"
    ckpt_root: str = "./checkpoints"
    save_best: bool = True

    # --- ERF logging ---
    erf_epochs: Tuple[int, ...] = (1, 10, 20, 30, 50)  # 마지막 epoch까지 포함
    erf_num_batches: int = 2

In [30]:
# ============================
# 3. Utils (seed, dirs, metrics, ERF, logging)
# ============================

def set_seed(seed: int) -> None:
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # NOTE: deterministic can slow down; keep for reproducibility
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def make_run_dir(cfg: Config) -> str:
    ts = time.strftime("%Y%m%d-%H%M%S")
    run_name = f"{cfg.exp_name}/{cfg.variant}/seed{cfg.seed}/{ts}"
    run_dir = os.path.join(cfg.log_root, run_name)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir


def make_ckpt_path(cfg: Config) -> str:
    ckpt_dir = os.path.join(cfg.ckpt_root, cfg.exp_name, cfg.variant, f"seed{cfg.seed}")
    os.makedirs(ckpt_dir, exist_ok=True)
    return os.path.join(ckpt_dir, "best.pth")


@torch.no_grad()
def accuracy_top1(logits: torch.Tensor, targets: torch.Tensor) -> float:
    preds = torch.argmax(logits, dim=1)
    return (preds == targets).float().mean().item()


def count_params(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def compute_single_cell_erf(
    model: nn.Module,
    loader: DataLoader,
    device: str,
    layer_name: str = "layer4",
    cell_xy: tuple = (0, 0),
    channel: int = 0,
    num_batches: int = 1,
) -> torch.Tensor:
    """
    Compute single-cell ERF:
    gradient of ONE spatial cell (x,y,channel) in an intermediate layer
    w.r.t input image.
    Output: (1, H, W) normalized heatmap
    """
    model.eval()

    activations = {}

    def hook_fn(_, __, output):
        activations["feat"] = output

    # register hook
    layer = dict(model.named_modules())[layer_name]
    hook = layer.register_forward_hook(hook_fn)

    agg = None
    seen = 0

    for b_idx, (images, _) in enumerate(loader):
        if b_idx >= num_batches:
            break

        images = images.to(device)
        images.requires_grad_(True)

        model.zero_grad()
        _ = model(images)

        feat = activations["feat"]  # (B, C, Hf, Wf)
        y, x = cell_xy

        target = feat[:, channel, y, x].sum()
        target.backward()

        grad = images.grad.detach().abs().mean(dim=1)  # (B, H, W)

        if agg is None:
            agg = grad.sum(dim=0)
        else:
            agg += grad.sum(dim=0)

        seen += images.size(0)

    hook.remove()

    heat = agg / max(seen, 1)
    heat = heat - heat.min()
    heat = heat / heat.max()

    return heat.unsqueeze(0)  # (1, H, W)

In [31]:
# ============================
# 4. Data Loader (CIFAR-100)
# ============================

def get_cifar100_dataloaders(cfg: Config) -> Tuple[DataLoader, DataLoader]:
    train_transform = T.Compose([
        T.RandomCrop(32, padding=4),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize(mean=(0.5071, 0.4867, 0.4408),
                    std=(0.2675, 0.2565, 0.2761)),
    ])
    test_transform = T.Compose([
        T.ToTensor(),
        T.Normalize(mean=(0.5071, 0.4867, 0.4408),
                    std=(0.2675, 0.2565, 0.2761)),
    ])

    train_dataset = torchvision.datasets.CIFAR100(
        root=cfg.data_dir, train=True, download=True, transform=train_transform
    )
    test_dataset = torchvision.datasets.CIFAR100(
        root=cfg.data_dir, train=False, download=True, transform=test_transform
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg.batch_size,
        shuffle=True,
        num_workers=cfg.num_workers,
        pin_memory=True,
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=cfg.batch_size,
        shuffle=False,
        num_workers=cfg.num_workers,
        pin_memory=True,
    )
    return train_loader, test_loader

In [32]:
# Helper functions

import torch
import torch.nn.functional as F

def shift_with_interpolation(
    x: torch.Tensor,
    dx: float,
    dy: float,
    padding_mode: str = "border",  # "border" = replicate edge
    align_corners: bool = True,
) -> torch.Tensor:
    """
    Shift feature map by (dx, dy) in pixel units using bilinear interpolation.
    - dx > 0: shift RIGHT
    - dy > 0: shift DOWN
    padding_mode:
      - "border" replicates boundary (closest valid pixel), like replicate padding.
      - "zeros" does zero padding.
    """
    B, C, H, W = x.shape

    # --- Build base grid in normalized coords [-1, 1] ---
    yy, xx = torch.meshgrid(
        torch.linspace(-1, 1, H, device=x.device, dtype=x.dtype),
        torch.linspace(-1, 1, W, device=x.device, dtype=x.dtype),
        indexing="ij",
    )
    grid = torch.stack((xx, yy), dim=-1).unsqueeze(0).repeat(B, 1, 1, 1)  # (B,H,W,2)

    # --- Convert pixel shift -> normalized shift ---
    # NOTE: with align_corners=True, step is 2/(W-1), 2/(H-1)
    denom_w = max(W - 1, 1)
    denom_h = max(H - 1, 1)
    shift_x = dx * 2.0 / denom_w
    shift_y = dy * 2.0 / denom_h

    # grid_sample samples input at "grid", so to shift output by +dx,
    # we sample input from grid - shift.
    grid[..., 0] -= shift_x
    grid[..., 1] -= shift_y

    return F.grid_sample(
        x,
        grid,
        mode="bilinear",
        padding_mode=padding_mode,  # "border" for replicate edges
        align_corners=align_corners,
    )

def shift_up(x):
    return F.pad(x, (0, 0, 0, 1), mode="replicate")[:, :, 1:, :]

def shift_down(x):
    return F.pad(x, (0, 0, 1, 0), mode="replicate")[:, :, :-1, :]

def shift_left(x):
    return F.pad(x, (0, 1, 0, 0), mode="replicate")[:, :, :, 1:]

def shift_right(x):
    return F.pad(x, (1, 0, 0, 0), mode="replicate")[:, :, :, :-1]

In [33]:
# ============================
# 5. Operators (Identity only for now)
# ============================

class IdentityOp(nn.Module):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x

# Random Shift 

class RandomShift4Way(nn.Module):
    """
    Random 4-way integer shift with replicate padding.

    - Each forward:
        * randomly permute channels
        * split into 4 equal groups
        * shift groups: up / down / left / right
    - Padding: replicate (NOT zero, NOT wrap)
    """

    def __init__(self):
        super().__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, C, H, W = x.shape
        g = C // 4

        # 1️⃣ random channel permutation
        perm = torch.randperm(C, device=x.device)

        idx_up    = perm[0*g : 1*g]
        idx_down  = perm[1*g : 2*g]
        idx_left  = perm[2*g : 3*g]
        idx_right = perm[3*g : 4*g]
        idx_rest  = perm[4*g :]

        out = x.clone()

        # 2️⃣ shift UP (replicate bottom row)
        if idx_up.numel() > 0:
            tmp = x[:, idx_up]
            tmp = F.pad(tmp, (0, 0, 0, 1), mode="replicate")
            out[:, idx_up] = tmp[:, :, 1:, :]

        # 3️⃣ shift DOWN (replicate top row)
        if idx_down.numel() > 0:
            tmp = x[:, idx_down]
            tmp = F.pad(tmp, (0, 0, 1, 0), mode="replicate")
            out[:, idx_down] = tmp[:, :, :-1, :]

        # 4️⃣ shift LEFT (replicate right column)
        if idx_left.numel() > 0:
            tmp = x[:, idx_left]
            tmp = F.pad(tmp, (0, 1, 0, 0), mode="replicate")
            out[:, idx_left] = tmp[:, :, :, 1:]

        # 5️⃣ shift RIGHT (replicate left column)
        if idx_right.numel() > 0:
            tmp = x[:, idx_right]
            tmp = F.pad(tmp, (1, 0, 0, 0), mode="replicate")
            out[:, idx_right] = tmp[:, :, :, :-1]

        # 6️⃣ leftover channels: identity
        if idx_rest.numel() > 0:
            out[:, idx_rest] = x[:, idx_rest]

        return out

# Deterministic Shift

class DeterministicShift4Way(nn.Module):
    """
    Deterministic 4-way integer shift with replicate padding.

    - Channels are split deterministically:
        [0:g]     -> up
        [g:2g]    -> down
        [2g:3g]   -> left
        [3g:4g]   -> right
    - Padding: replicate
    """

    def __init__(self):
        super().__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, C, H, W = x.shape
        g = C // 4

        idx_up    = slice(0*g, 1*g)
        idx_down  = slice(1*g, 2*g)
        idx_left  = slice(2*g, 3*g)
        idx_right = slice(3*g, 4*g)
        idx_rest  = slice(4*g, C)

        out = x.clone()

        # shift UP
        if g > 0:
            tmp = x[:, idx_up]
            tmp = F.pad(tmp, (0, 0, 0, 1), mode="replicate")
            out[:, idx_up] = tmp[:, :, 1:, :]

        # shift DOWN
        if g > 0:
            tmp = x[:, idx_down]
            tmp = F.pad(tmp, (0, 0, 1, 0), mode="replicate")
            out[:, idx_down] = tmp[:, :, :-1, :]

        # shift LEFT
        if g > 0:
            tmp = x[:, idx_left]
            tmp = F.pad(tmp, (0, 1, 0, 0), mode="replicate")
            out[:, idx_left] = tmp[:, :, :, 1:]

        # shift RIGHT
        if g > 0:
            tmp = x[:, idx_right]
            tmp = F.pad(tmp, (1, 0, 0, 0), mode="replicate")
            out[:, idx_right] = tmp[:, :, :, :-1]

        # leftover channels: identity
        if 4 * g < C:
            out[:, idx_rest] = x[:, idx_rest]

        return out

# Random Shift with Identity (Partial Random Shift)

class RandomShift4WayPartial(nn.Module):
    """
    Random 4-way integer shift with fixed ratio + identity channels.

    Example (C=64, n_shift=10):
        - 10 up
        - 10 down
        - 10 left
        - 10 right
        - 24 identity

    Each forward:
        * randomly permute channels
        * apply deterministic split sizes
    """

    def __init__(self, n_shift: int = 10):
        super().__init__()
        self.n_shift = n_shift  # per direction

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, C, H, W = x.shape
        k = self.n_shift

        assert 4 * k <= C, "Not enough channels for requested shift split."

        # 1️⃣ random channel permutation
        perm = torch.randperm(C, device=x.device)

        idx_up    = perm[0*k : 1*k]
        idx_down  = perm[1*k : 2*k]
        idx_left  = perm[2*k : 3*k]
        idx_right = perm[3*k : 4*k]
        idx_rest  = perm[4*k :]

        out = x.clone()

        # 2️⃣ shift UP (replicate bottom row)
        if idx_up.numel() > 0:
            tmp = F.pad(x[:, idx_up], (0, 0, 0, 1), mode="replicate")
            out[:, idx_up] = tmp[:, :, 1:, :]

        # 3️⃣ shift DOWN (replicate top row)
        if idx_down.numel() > 0:
            tmp = F.pad(x[:, idx_down], (0, 0, 1, 0), mode="replicate")
            out[:, idx_down] = tmp[:, :, :-1, :]

        # 4️⃣ shift LEFT (replicate right column)
        if idx_left.numel() > 0:
            tmp = F.pad(x[:, idx_left], (0, 1, 0, 0), mode="replicate")
            out[:, idx_left] = tmp[:, :, :, 1:]

        # 5️⃣ shift RIGHT (replicate left column)
        if idx_right.numel() > 0:
            tmp = F.pad(x[:, idx_right], (1, 0, 0, 0), mode="replicate")
            out[:, idx_right] = tmp[:, :, :, :-1]

        # 6️⃣ identity channels
        if idx_rest.numel() > 0:
            out[:, idx_rest] = x[:, idx_rest]

        return out

# Real value Random Shift with Interpolation

class RandomShift4WayFloat(nn.Module):
    """
    Random 4-way *float* shift (bilinear interpolation).

    Example (C=64):
        - 16 up
        - 16 down
        - 16 left
        - 16 right

    shift_px: float (e.g., 0.5)
    padding_mode="border" replicates boundary values.
    """

    def __init__(self, shift_px: float = 0.5):
        super().__init__()
        self.shift_px = float(shift_px)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, C, H, W = x.shape
        assert C % 4 == 0, "Channel dimension must be divisible by 4."

        g = C // 4
        perm = torch.randperm(C, device=x.device)

        idx_up    = perm[0*g : 1*g]
        idx_down  = perm[1*g : 2*g]
        idx_left  = perm[2*g : 3*g]
        idx_right = perm[3*g : 4*g]

        out = x.clone()
        s = self.shift_px

        out[:, idx_up]    = shift_with_interpolation(x[:, idx_up],    dx=0.0, dy=-s)
        out[:, idx_down]  = shift_with_interpolation(x[:, idx_down],  dx=0.0, dy=+s)
        out[:, idx_left]  = shift_with_interpolation(x[:, idx_left],  dx=-s,  dy=0.0)
        out[:, idx_right] = shift_with_interpolation(x[:, idx_right], dx=+s,  dy=0.0)

        return out

class LearnableShift4Way(nn.Module):
    """
    Channel-wise learnable soft shift (NO conv, NO interpolation)

    y_c = sum_d softmax(logits[c])_d * S_d(x_c)

    directions:
      [up, down, left, right, identity]
    """

    def __init__(
        self,
        num_channels: int = 64,
        temperature: float = 1.0,
        init_identity_bias: float = 0.3,   # 🔹 거의 0에 가까운 bias
        init_noise_std: float = 1e-3,      # 🔹 symmetry breaking
    ):
        super().__init__()
        self.num_channels = num_channels
        self.temperature = temperature

        # logits: (C, 5)
        self.logits = nn.Parameter(torch.zeros(num_channels, 5))

        # small identity bias
        self.logits.data[:, 4] = init_identity_bias

        # tiny noise to avoid symmetric collapse
        self.logits.data += init_noise_std * torch.randn_like(self.logits.data)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, C, H, W = x.shape
        assert C == self.num_channels, "Channel mismatch"

        # soft direction weights
        probs = F.softmax(self.logits / self.temperature, dim=-1)  # (C,5)

        # shifted versions
        x_up    = shift_up(x)
        x_down  = shift_down(x)
        x_left  = shift_left(x)
        x_right = shift_right(x)
        x_id    = x

        stacked = torch.stack(
            [x_up, x_down, x_left, x_right, x_id],
            dim=2,   # (B, C, 5, H, W)
        )

        probs = probs.view(1, C, 5, 1, 1)
        out = (stacked * probs).sum(dim=2)

        return out

In [34]:
# ============================
# 6. Model Definition (CIFAR-ResNet18 + operator plug-in)
# ============================

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_ch: int, out_ch: int, stride: int = 1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_ch)

        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_ch)

        self.relu = nn.ReLU(inplace=True)

        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.relu(out + identity)
        return out


class ResNetCIFAR18(nn.Module):
    """
    CIFAR ResNet-18:
      Stem: 3x3 conv, stride=1 (no 7x7, no maxpool)
      Stages: [2,2,2,2] BasicBlocks
    Operator plug-in:
      - stage1_pre_op is applied right BEFORE Stage1
      - stage1_post_op is applied right AFTER Stage1
    """
    def __init__(
        self,
        num_classes: int = 100,
        stage1_pre_op: Optional[nn.Module] = None,
        stage1_post_op: Optional[nn.Module] = None,
    ):
        super().__init__()
        self.in_ch = 64

        # Stem
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        # Operators (default Identity)
        self.stage1_pre_op = stage1_pre_op if stage1_pre_op is not None else IdentityOp()
        self.stage1_post_op = stage1_post_op if stage1_post_op is not None else IdentityOp()

        # Stages
        self.layer1 = self._make_layer(64,  2, stride=1)  # Stage1: 32x32
        self.layer2 = self._make_layer(128, 2, stride=2)  # Stage2: 16x16
        self.layer3 = self._make_layer(256, 2, stride=2)  # Stage3:  8x8
        self.layer4 = self._make_layer(512, 2, stride=2)  # Stage4:  4x4

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

        self._init_weights()

    def _make_layer(self, out_ch: int, num_blocks: int, stride: int) -> nn.Sequential:
        strides = [stride] + [1] * (num_blocks - 1)
        blocks = []
        for s in strides:
            blocks.append(BasicBlock(self.in_ch, out_ch, stride=s))
            self.in_ch = out_ch
        return nn.Sequential(*blocks)

    def _init_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1.0)
                nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0.0, 0.01)
                nn.init.constant_(m.bias, 0.0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Stem
        x = self.relu(self.bn1(self.conv1(x)))   # (B,64,32,32)

        # Stage1 (intervention point)
        x = self.stage1_pre_op(x)
        x = self.layer1(x)                       # (B,64,32,32)
        x = self.stage1_post_op(x)

        # Rest
        x = self.layer2(x)                       # (B,128,16,16)
        x = self.layer3(x)                       # (B,256, 8, 8)
        x = self.layer4(x)                       # (B,512, 4, 4)

        x = self.avgpool(x)                      # (B,512,1,1)
        x = torch.flatten(x, 1)
        x = self.fc(x)                           # (B,100)
        return x


def build_model(cfg: Config) -> nn.Module:
    """
    Build model according to cfg.variant.
    ONLY this function controls operator choice.
    """

    if cfg.variant == "baseline":
        pre_op  = IdentityOp()
        post_op = IdentityOp()

    elif cfg.variant == "shift_int":
        pre_op  = RandomShift4Way()
        post_op = IdentityOp()

    elif cfg.variant == "shift_int_partial":
        pre_op  = RandomShift4WayPartial(n_shift=10)
        post_op = IdentityOp()

    elif cfg.variant == "shift_det":
        pre_op  = DeterministicShift4Way()
        post_op = IdentityOp()

    elif cfg.variant == "shift_float_05":
        pre_op  = RandomShift4WayFloat(shift_px=0.5)
        post_op = IdentityOp()

    elif cfg.variant == "shift_learnable":
        pre_op = LearnableShift4Way(
            num_channels=64,
            temperature=1.0,
            init_identity_bias=0.1,
        )
        post_op = IdentityOp()

    else:
        raise ValueError(f"Unknown variant: {cfg.variant}")

    return ResNetCIFAR18(
        num_classes=cfg.num_classes,
        stage1_pre_op=pre_op,
        stage1_post_op=post_op,
    )

In [35]:
# ============================
# 7. Training & Evaluation Loop
# ============================

def train_one_epoch(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    optimizer: optim.Optimizer,
    device: str,
) -> Tuple[float, float]:
    model.train()
    total_loss = 0.0
    total_acc = 0.0
    n = 0

    for images, targets in loader:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        logits = model(images)
        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()

        bsz = images.size(0)
        total_loss += loss.item() * bsz
        total_acc += accuracy_top1(logits, targets) * bsz
        n += bsz

    return total_loss / n, total_acc / n


@torch.no_grad()
def evaluate(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    device: str,
) -> Tuple[float, float]:
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    n = 0

    for images, targets in loader:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        logits = model(images)
        loss = criterion(logits, targets)

        bsz = images.size(0)
        total_loss += loss.item() * bsz
        total_acc += accuracy_top1(logits, targets) * bsz
        n += bsz

    return total_loss / n, total_acc / n

In [36]:
# ============================
# 8. Main (Run one experiment)
# ============================

def main(cfg: Config) -> None:
    set_seed(cfg.seed)
    device = cfg.device
    print("Using device:", device)

    # dirs
    run_dir = make_run_dir(cfg)
    ckpt_path = make_ckpt_path(cfg)
    writer = SummaryWriter(log_dir=run_dir)

    # data
    train_loader, test_loader = get_cifar100_dataloaders(cfg)

    # model
    model = build_model(cfg).to(device)
    n_params = count_params(model)
    print(f"Model params: {n_params:,}")

    # loss/opt/sched
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(
        model.parameters(),
        lr=cfg.lr,
        momentum=cfg.momentum,
        weight_decay=cfg.weight_decay,
    )
    scheduler = optim.lr_scheduler.MultiStepLR(
        optimizer, milestones=[60, 90], gamma=0.2
    )

    # log fixed config
    writer.add_text("config/exp_name", cfg.exp_name)
    writer.add_text("config/variant", cfg.variant)
    writer.add_scalar("model/params", n_params, 0)

    best_acc = 0.0

    for epoch in range(1, cfg.num_epochs + 1):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        scheduler.step()

        # console
        print(
            f"Epoch [{epoch:03d}/{cfg.num_epochs}] "
            f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}% "
            f"|| Test Loss: {test_loss:.4f} | Test Acc: {test_acc*100:.2f}%"
        )

        # tensorboard scalars
        writer.add_scalar("loss/train", train_loss, epoch)
        writer.add_scalar("loss/test", test_loss, epoch)
        writer.add_scalar("acc/train", train_acc, epoch)
        writer.add_scalar("acc/test", test_acc, epoch)
        writer.add_scalar("lr", optimizer.param_groups[0]["lr"], epoch)

        # ERF snapshots
        if epoch in cfg.erf_epochs:
            heat = compute_single_cell_erf(
                model=model,
                loader=test_loader,
                device=device,
                layer_name="layer4",
                cell_xy=(2, 2),     # 중앙 cell
                channel=0,          # 고정 channel
                num_batches=cfg.erf_num_batches,
            )
            
            writer.add_image(
                f"erf/layer4_cell(2,2)_epoch{epoch}",
                heat,
                global_step=epoch,
            )

        # save best
        if cfg.save_best and test_acc > best_acc:
            best_acc = test_acc
            torch.save(
                {
                    "epoch": epoch,
                    "model_state": model.state_dict(),
                    "optimizer_state": optimizer.state_dict(),
                    "best_acc": best_acc,
                    "cfg": cfg.__dict__,
                },
                ckpt_path,
            )
            print(f"  -> New best: {best_acc*100:.2f}% | Saved: {ckpt_path}")

    writer.close()
    print(f"Done. Best test acc: {best_acc*100:.2f}%")
    print(f"TensorBoard log dir: {run_dir}")
    print(f"Best ckpt path: {ckpt_path}")

In [20]:
#if __name__ == "__main__":
#    cfg = Config()
#    # baseline run:
#    cfg.variant = "baseline"
#    main(cfg)

Using device: cuda
Model params: 11,220,132
Epoch [001/120] Train Loss: 3.9624 | Train Acc: 8.72% || Test Loss: 3.5956 | Test Acc: 14.07%
  -> New best: 14.07% | Saved: ./checkpoints/cifar100_resnet18/baseline/seed42/best.pth
Epoch [002/120] Train Loss: 3.3127 | Train Acc: 18.89% || Test Loss: 3.1484 | Test Acc: 22.63%
  -> New best: 22.63% | Saved: ./checkpoints/cifar100_resnet18/baseline/seed42/best.pth
Epoch [003/120] Train Loss: 2.7318 | Train Acc: 29.69% || Test Loss: 2.7154 | Test Acc: 31.69%
  -> New best: 31.69% | Saved: ./checkpoints/cifar100_resnet18/baseline/seed42/best.pth
Epoch [004/120] Train Loss: 2.2953 | Train Acc: 38.70% || Test Loss: 2.2868 | Test Acc: 39.16%
  -> New best: 39.16% | Saved: ./checkpoints/cifar100_resnet18/baseline/seed42/best.pth
Epoch [005/120] Train Loss: 2.0063 | Train Acc: 44.84% || Test Loss: 2.1470 | Test Acc: 42.25%
  -> New best: 42.25% | Saved: ./checkpoints/cifar100_resnet18/baseline/seed42/best.pth
Epoch [006/120] Train Loss: 1.8214 | Train

In [26]:
# if __name__ == "__main__":
#    cfg = Config()
#    cfg.variant = "shift_int"
#    main(cfg)

Using device: cuda
Model params: 11,220,132
Epoch [001/120] Train Loss: 4.0010 | Train Acc: 8.18% || Test Loss: 3.6481 | Test Acc: 13.31%
  -> New best: 13.31% | Saved: ./checkpoints/cifar100_resnet18/shift_int/seed42/best.pth
Epoch [002/120] Train Loss: 3.4072 | Train Acc: 17.20% || Test Loss: 3.2011 | Test Acc: 21.94%
  -> New best: 21.94% | Saved: ./checkpoints/cifar100_resnet18/shift_int/seed42/best.pth
Epoch [003/120] Train Loss: 2.9495 | Train Acc: 25.98% || Test Loss: 2.9408 | Test Acc: 26.21%
  -> New best: 26.21% | Saved: ./checkpoints/cifar100_resnet18/shift_int/seed42/best.pth
Epoch [004/120] Train Loss: 2.5471 | Train Acc: 33.83% || Test Loss: 2.7089 | Test Acc: 32.02%
  -> New best: 32.02% | Saved: ./checkpoints/cifar100_resnet18/shift_int/seed42/best.pth
Epoch [005/120] Train Loss: 2.2546 | Train Acc: 39.69% || Test Loss: 2.5098 | Test Acc: 35.00%
  -> New best: 35.00% | Saved: ./checkpoints/cifar100_resnet18/shift_int/seed42/best.pth
Epoch [006/120] Train Loss: 2.0546 | 

In [32]:
# if __name__ == "__main__":
#    cfg = Config()
#    cfg.variant = "shift_int_det"
#    main(cfg)

Using device: cuda
Model params: 11,220,132
Epoch [001/120] Train Loss: 3.9757 | Train Acc: 8.58% || Test Loss: 3.6705 | Test Acc: 13.40%
  -> New best: 13.40% | Saved: ./checkpoints/cifar100_resnet18/shift_int_det/seed42/best.pth
Epoch [002/120] Train Loss: 3.3466 | Train Acc: 18.48% || Test Loss: 3.1485 | Test Acc: 22.96%
  -> New best: 22.96% | Saved: ./checkpoints/cifar100_resnet18/shift_int_det/seed42/best.pth
Epoch [003/120] Train Loss: 2.7829 | Train Acc: 29.00% || Test Loss: 2.7922 | Test Acc: 30.18%
  -> New best: 30.18% | Saved: ./checkpoints/cifar100_resnet18/shift_int_det/seed42/best.pth
Epoch [004/120] Train Loss: 2.3302 | Train Acc: 37.90% || Test Loss: 2.5441 | Test Acc: 35.48%
  -> New best: 35.48% | Saved: ./checkpoints/cifar100_resnet18/shift_int_det/seed42/best.pth
Epoch [005/120] Train Loss: 2.0365 | Train Acc: 43.92% || Test Loss: 2.2332 | Test Acc: 40.29%
  -> New best: 40.29% | Saved: ./checkpoints/cifar100_resnet18/shift_int_det/seed42/best.pth
Epoch [006/120] T

In [42]:
# if __name__ == "__main__":
#     cfg = Config()
#     cfg.variant = "shift_int_partial"  # 여기만 바꾸면 됨
#     main(cfg)

Using device: cuda
Model params: 11,220,132
Epoch [001/120] Train Loss: 3.9888 | Train Acc: 8.56% || Test Loss: 3.6558 | Test Acc: 13.31%
  -> New best: 13.31% | Saved: ./checkpoints/cifar100_resnet18/shift_int_partial/seed42/best.pth
Epoch [002/120] Train Loss: 3.3645 | Train Acc: 18.01% || Test Loss: 3.1511 | Test Acc: 21.83%
  -> New best: 21.83% | Saved: ./checkpoints/cifar100_resnet18/shift_int_partial/seed42/best.pth
Epoch [003/120] Train Loss: 2.8475 | Train Acc: 27.60% || Test Loss: 2.8732 | Test Acc: 28.47%
  -> New best: 28.47% | Saved: ./checkpoints/cifar100_resnet18/shift_int_partial/seed42/best.pth
Epoch [004/120] Train Loss: 2.4168 | Train Acc: 36.31% || Test Loss: 2.5427 | Test Acc: 34.58%
  -> New best: 34.58% | Saved: ./checkpoints/cifar100_resnet18/shift_int_partial/seed42/best.pth
Epoch [005/120] Train Loss: 2.1213 | Train Acc: 42.57% || Test Loss: 2.2293 | Test Acc: 41.09%
  -> New best: 41.09% | Saved: ./checkpoints/cifar100_resnet18/shift_int_partial/seed42/best.p

In [48]:
# if __name__ == "__main__":
#    cfg = Config()
#    cfg.variant = "shift_float_05"  # 여기만 바꾸면 됨
#    main(cfg)

Using device: cuda
Model params: 11,220,132
Epoch [001/120] Train Loss: 3.9798 | Train Acc: 8.60% || Test Loss: 3.6227 | Test Acc: 13.80%
  -> New best: 13.80% | Saved: ./checkpoints/cifar100_resnet18/shift_float_05/seed42/best.pth
Epoch [002/120] Train Loss: 3.3675 | Train Acc: 18.24% || Test Loss: 3.1263 | Test Acc: 22.61%
  -> New best: 22.61% | Saved: ./checkpoints/cifar100_resnet18/shift_float_05/seed42/best.pth
Epoch [003/120] Train Loss: 2.8514 | Train Acc: 27.51% || Test Loss: 2.8069 | Test Acc: 29.10%
  -> New best: 29.10% | Saved: ./checkpoints/cifar100_resnet18/shift_float_05/seed42/best.pth
Epoch [004/120] Train Loss: 2.3922 | Train Acc: 36.66% || Test Loss: 2.5428 | Test Acc: 34.92%
  -> New best: 34.92% | Saved: ./checkpoints/cifar100_resnet18/shift_float_05/seed42/best.pth
Epoch [005/120] Train Loss: 2.0881 | Train Acc: 43.32% || Test Loss: 2.2481 | Test Acc: 40.54%
  -> New best: 40.54% | Saved: ./checkpoints/cifar100_resnet18/shift_float_05/seed42/best.pth
Epoch [006/1

In [20]:
if __name__ == "__main__":
    variants = [
        "baseline",
        "shift_int",
        "shift_det",
        "shift_float_05",
    ]
    seeds = [8, 9, 10]

    for variant in variants:
        for seed in seeds:
            cfg = Config()
            cfg.variant = variant
            cfg.seed = seed
            main(cfg)

Using device: cuda
Model params: 11,220,132
Epoch [001/120] Train Loss: 4.0036 | Train Acc: 8.12% || Test Loss: 3.7052 | Test Acc: 12.09%
  -> New best: 12.09% | Saved: ./checkpoints/cifar100_resnet18/baseline/seed8/best.pth
Epoch [002/120] Train Loss: 3.3793 | Train Acc: 17.84% || Test Loss: 3.1463 | Test Acc: 22.97%
  -> New best: 22.97% | Saved: ./checkpoints/cifar100_resnet18/baseline/seed8/best.pth
Epoch [003/120] Train Loss: 2.8306 | Train Acc: 27.53% || Test Loss: 2.6701 | Test Acc: 30.42%
  -> New best: 30.42% | Saved: ./checkpoints/cifar100_resnet18/baseline/seed8/best.pth
Epoch [004/120] Train Loss: 2.3482 | Train Acc: 37.68% || Test Loss: 2.4740 | Test Acc: 36.51%
  -> New best: 36.51% | Saved: ./checkpoints/cifar100_resnet18/baseline/seed8/best.pth
Epoch [005/120] Train Loss: 2.0454 | Train Acc: 44.47% || Test Loss: 2.1512 | Test Acc: 42.31%
  -> New best: 42.31% | Saved: ./checkpoints/cifar100_resnet18/baseline/seed8/best.pth
Epoch [006/120] Train Loss: 1.8404 | Train Acc:

In [40]:
# runs 폴더가 있는 위치에서 실행
!tar -czf runs_tb.tar.gz runs

In [38]:
if __name__ == "__main__":
    seeds = [8, 9, 10]

    for seed in seeds:
        cfg = Config()
        cfg.variant = "shift_learnable"   # ✅ learnable soft shift
        cfg.seed = seed
        main(cfg)

Using device: cuda
Model params: 11,220,452
Epoch [001/120] Train Loss: 3.9791 | Train Acc: 8.42% || Test Loss: 3.6280 | Test Acc: 12.65%
  -> New best: 12.65% | Saved: ./checkpoints/cifar100_resnet18/shift_learnable/seed8/best.pth
Epoch [002/120] Train Loss: 3.3688 | Train Acc: 17.89% || Test Loss: 3.1911 | Test Acc: 21.42%
  -> New best: 21.42% | Saved: ./checkpoints/cifar100_resnet18/shift_learnable/seed8/best.pth
Epoch [003/120] Train Loss: 2.8927 | Train Acc: 26.78% || Test Loss: 2.8876 | Test Acc: 27.71%
  -> New best: 27.71% | Saved: ./checkpoints/cifar100_resnet18/shift_learnable/seed8/best.pth
Epoch [004/120] Train Loss: 2.4640 | Train Acc: 35.43% || Test Loss: 2.5550 | Test Acc: 34.38%
  -> New best: 34.38% | Saved: ./checkpoints/cifar100_resnet18/shift_learnable/seed8/best.pth
Epoch [005/120] Train Loss: 2.1537 | Train Acc: 42.01% || Test Loss: 2.1222 | Test Acc: 42.90%
  -> New best: 42.90% | Saved: ./checkpoints/cifar100_resnet18/shift_learnable/seed8/best.pth
Epoch [006/1

In [39]:
import os
import torch
import numpy as np
import pandas as pd

exp_name = "cifar100_resnet18"
seeds = [8, 9, 10, 42]
ckpt_root = "./checkpoints"

# 🔑 표에 쓸 model 이름 → 실제 폴더 이름을 결정하는 함수
def resolve_variant_dir(model_name, seed):
    if model_name == "shift_det" and seed == 42:
        return "shift_int_det"   # 예외 처리
    return model_name

models = [
    "baseline",
    "shift_int",
    "shift_det",
    "shift_float_05",
    "shift_learnable",   # ✅ 추가
]

rows = []

for model_name in models:
    accs = []
    row = {"model": model_name}

    for seed in seeds:
        variant_dir = resolve_variant_dir(model_name, seed)

        ckpt_path = os.path.join(
            ckpt_root,
            exp_name,
            variant_dir,
            f"seed{seed}",
            "best.pth",
        )

        if not os.path.exists(ckpt_path):
            print(f"⚠️ Missing: {ckpt_path}")
            row[f"seed{seed}"] = np.nan
            continue

        ckpt = torch.load(ckpt_path, map_location="cpu")
        acc = ckpt["best_acc"] * 100.0  # percent

        row[f"seed{seed}"] = acc
        accs.append(acc)

    row["mean"] = np.mean(accs)
    row["std"] = np.std(accs, ddof=1)
    rows.append(row)

df = pd.DataFrame(rows)
df = df[["model"] + [f"seed{s}" for s in seeds] + ["mean", "std"]]
df

,model,seed8,seed9,seed10,seed42,mean,std
0,baseline,77.82,77.65,77.54,77.45,77.6150,0.159269
1,shift_int,76.40,76.01,75.98,76.28,76.1675,0.205487
2,shift_det,76.90,77.14,77.22,77.47,77.1825,0.235000
3,shift_float_05,76.70,76.84,76.19,76.85,76.6450,0.310966
4,shift_learnable,76.79,76.90,77.07,77.27,77.0075,0.209503


In [37]:
#if __name__ == "__main__":
#    cfg = Config()
#    cfg.variant = "shift_learnable"   # ✅ learnable soft shift
#    main(cfg)

Using device: cuda
Model params: 11,220,452
Epoch [001/120] Train Loss: 3.9867 | Train Acc: 8.20% || Test Loss: 3.6232 | Test Acc: 13.80%
  -> New best: 13.80% | Saved: ./checkpoints/cifar100_resnet18/shift_learnable/seed42/best.pth
Epoch [002/120] Train Loss: 3.3709 | Train Acc: 17.86% || Test Loss: 3.1405 | Test Acc: 21.84%
  -> New best: 21.84% | Saved: ./checkpoints/cifar100_resnet18/shift_learnable/seed42/best.pth
Epoch [003/120] Train Loss: 2.8903 | Train Acc: 27.07% || Test Loss: 2.7706 | Test Acc: 28.95%
  -> New best: 28.95% | Saved: ./checkpoints/cifar100_resnet18/shift_learnable/seed42/best.pth
Epoch [004/120] Train Loss: 2.4650 | Train Acc: 35.32% || Test Loss: 2.3733 | Test Acc: 37.43%
  -> New best: 37.43% | Saved: ./checkpoints/cifar100_resnet18/shift_learnable/seed42/best.pth
Epoch [005/120] Train Loss: 2.1480 | Train Acc: 41.93% || Test Loss: 2.2537 | Test Acc: 41.13%
  -> New best: 41.13% | Saved: ./checkpoints/cifar100_resnet18/shift_learnable/seed42/best.pth
Epoch [